# Exercises for Modelling Primer

## Code to set-up Google Colab environment

### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Clone repository of exercises

In [ ]:
%%bash
cd /content/drive/MyDrive/
mkdir -p cdt_training
cd cdt_training
git clone https://github.com/AtomisticSimulationOfMaterials/MaterialsModellingPrimer.git

### Install the lammps molecular dynamics software


In [ ]:
# Install lammps
!pip install lammps[mpi]
# Install package to assist in parsing lammps log files
!pip install lammps-logfile

### Load some modules 

In [ ]:
import numpy as np
from scipy import optimize
import os
import shutil
import subprocess
from contextlib import chdir
import lammps_logfile
import matplotlib.pyplot as plt
%matplotlib inline

### Define a useful function


In [ ]:
# Define a function to replace text in base input file for lammps
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

## Exercise 1: Determine the lattice parameter of perfect fcc crystal 

### Change to the correct directory

In [ ]:
%cd /content/drive/MyDrive/cdt_training/MaterialsModellingPrimer/01-LatticeParameters

### Define a useful function

This cell defines a function which takes some details of a supercell and writes them to a file suitable for use with Lammps. You do not need to worry about the details for now, but note the form of the function definition:

    def write_lammps(supercell, atom_pos, atom_type=None, filename='lammps.txt', num_types=1):
    
The function takes the following arguments:
- `supercell`: this is a 3x3 array with the row being the vectors defining the edges of the supercell;
- `atom_pos`: an Nx3 array givein the cartesian coordinates of the atoms in the supercell;
- `atom_type`: is an optional array of size N giving the type identifier for each atom as an integer. By default each atom is assigned the same type of 1;
- `filename`: the name of the file to write the output to (optional);
- `num_types`: the number of distinct atom types in the supercell. Optional, defaults to 1.

Note that this isn't particularly well written code, but it does the job.

In [ ]:
# Define a function to write out a file in the correct format for lammps
def write_lammps(supercell, atom_pos, atom_type=None, filename='lammps.txt', num_types=1):
    """Write out a supercell in Lammps format"""
    fo = open(filename,'w')
    header = '#Lammps coordinate file'
    fo.write(header)
    fo.write('\n')
    fo.write(str(np.shape(atom_pos)[0]) + ' atoms\n')
    fo.write('\n')
    fo.write(str(num_types) + ' atom types\n')
    fo.write('\n')
    fo.write('0.0 ' + str(supercell[0,0]) + ' xlo xhi\n')
    fo.write('0.0 ' + str(supercell[1,1]) + ' ylo yhi\n')
    fo.write('0.0 ' + str(supercell[2,2]) + ' zlo zhi\n')
    if abs(supercell[1,0]) + abs(supercell[2,0]) + abs(supercell[2,1]) > 1e-3:
        fo.write(str(supercell[1,0]) + ' ' + str(supercell[2,0]) + ' ' + str(supercell[2,1]) + ' xy xz yz\n')
    fo.write('\n')
    fo.write('Atoms\n')
    fo.write('\n')
    count = 1
    for i in range(np.shape(atom_pos)[0]):
        fo.write(str(count) + ' ')
        if atom_type is not None: 
            fo.write(str(int(atom_type[i])) + ' ') 
        else:
            fo.write('1 ') 
        fo.write(str(atom_pos[i,0]) + ' ' + str(atom_pos[i,1]) + ' ' + str(atom_pos[i,2]) + '\n')
        count = count + 1
    fo.flush()
    fo.close()
    return

### Define names and properties of available potentials


In [ ]:
pair_styles = ['eam/alloy', 'eam/fs', 'eam/alloy', 'eam/alloy', 'eam/alloy', 'eam/alloy']
potential_files = ['Pot1.set', 'Pot2.eam.fs', 'Pot3.eam.alloy', 'Pot4.eam.alloy', 'Pot5.eam.alloy', 'Pot6.eam.fs']

### Select which potential to use

Add a value 0,1,2,3,4 or 5 to select which potential to use (check you have selected the correct one).

In [ ]:
potential_index = 

potential_file = potential_files[potential_index]
pair_style = pair_styles[potential_index]
print('You have selected potential: ', potential_file)
print('This potential is of style: ', pair_style)

### Set up a simulation folder

In [ ]:
# ------------- Set up a folder for the simulation
path = './Simulations/' + 'potential_' + str(potential_index+1) +  '/'
if not os.path.exists(path):
    os.makedirs(path, mode=0o777)

### Build the crystal 

This code cell actually builds the crystal and stores in an array

In [ ]:
# ------------- Set up the crystal definition
# Set initial lattice parameters to experimental values

a = 4.2   # Rough value of lattice parameter. Relaxation will do the rest

# Set up unit cell specification and basis
cell = np.array([
    [1.0,0.0,0.0],
    [0.0,1.0,0.0],
    [0.0,0.0,1.0]])
motif = (np.array([
    [0.0, 0.0, 0.0],
    [0.5, 0.5, 0.0],
    [0.5, 0.0, 0.5],
    [0.0, 0.5, 0.5]]))
motif_size = 4

# ------------- Set size of simulation
# Set size of supercell (number of unit cells in each direction)
block_size = np.array([10,10,10])
# Calculate vectors defining supercell box
supercell = np.zeros((3,3), dtype=float)
for s in range(3):
    supercell[s,:] = a * block_size[s] * cell[s,:]

# ------------- Calculate atomic coordinates in supercell
# Set up empty list to hold coordinates
r = []
# Loop over all unit cells and all atoms in motif and add atom positions to a list
for i in range(block_size[0]):
    for j in range(block_size[1]):
        for k in range(block_size[2]):
            for p in range(motif_size):
                pos = a * ( (i + motif[p,0])*cell[0,:] + (j + motif[p,1])*cell[1,:] + (k + motif[p,2])*cell[2,:] )
                r.append(pos.tolist())
# Get number of atoms in total
num_atoms = len(r)
# Convert list of atoms to an array
r = np.array(r)

### Write the crystal structure to a file
write out a suitable input file using the `write_lammps()` function defined above.

In [ ]:
# ------------- Write out the file using the function
write_lammps(supercell, r, filename=path + 'lammps.txt')

### Copy Lammps instruction file
Copy the file of instructions for lammps to the simulation folder, making suitable amendments

In [ ]:
# Create  a folder for simulation and copy in the empirical potential file
shutil.copy2('../Potentials/' + potential_file, path)

# Define values to replace in base input file
replacements = {
    'POTENTIAL':potential_file,
    'PAIRSTYLE':pair_style
    }
        
# Create a lammps file and use base file with string replacements to create unique variant
inputLammpsFile = 'in_base.lmps'
outputLammpsFile = path+'in.lmps'        
finLammps = open(inputLammpsFile, 'r').read()
foutLammps = open(outputLammpsFile, 'w')
out = replace_all(finLammps, replacements)
foutLammps.write(out)
foutLammps.close()

### Run the simulation


In [ ]:
lammps_executable = 'lmp'
lammps_command = '-in in.lmps'

with chdir(path):
    os.system(lammps_executable + ' ' + lammps_command)

### Check the results from the simulation 

These cells are for use after the simulation has run, to examine the output.

First read in data from the logfile

In [ ]:
log = lammps_logfile.File(path + 'log.lammps')

step = log.get("Step")
pe = log.get("PotEng")
length = log.get("Lx")
pressure = log.get("Pxx")

Now print out the final results

In [ ]:
print("Optimised lattice parameter = %7.6f angstrom" % (length[-1]/10))
print("Equilibrium energy per atom = %7.6f eV" % (pe[-1]/4000))

And store the final potential energy per atom and lattice parameter for later use

In [ ]:
E_eq = pe[-1]/4000
a_eq = length[-1]/10

We can also examine how the potential energy evolves as the system relaxes

In [ ]:
fig, ax = plt.subplots()
ax.plot(step, (pe-pe[-1])/4000)
ax.set(xlabel='Step', ylabel='Potential energy per atom (eV)', title='')
ax.grid()
fig.savefig(path + 'pe.png')
plt.show()

And the corresponding variation in the lattice parameter

In [ ]:
fig, ax = plt.subplots()
ax.plot(step, length/10)
ax.set(xlabel='Step', ylabel='Lattice parameter (Ang)', title='')
ax.grid()
fig.savefig(path + 'a.png')
plt.show()

This is how the pressure varies with lattice parameter (we started with an overestimate of the lattice parameter)

In [ ]:
fig, ax = plt.subplots()
ax.plot(length/10, pressure)
ax.set(xlabel='Latttice parameter (Ang)', ylabel='Pressure (bar)', title='')
ax.grid()
fig.savefig(path + 'pressure.png')
plt.show()

## Exercise 2: Finding the bulk modulus of model aluminium


### Change to the correct directory

In [ ]:
%cd /content/drive/MyDrive/cdt_training/MaterialsModellingPrimer/02-BulkModulus

### Build and run the simulations
The below code uses a loop over the values of the variable `scale` to create and run multiple simulations with different values for the lattice parameter. 

In [ ]:
# Define a function to replace text in base input file
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

scale = [0.980, 0.985, 0.990, 0.995, 
         1.000, 1.005, 1.010, 1.015, 1.020]     # Factors by which to scale lattice constant
ncells = [5]                                    # Number of unit cells in each direction

# Loop over values of unit cell size and number of unit cells in simulation
# Create a folder and lammps input files for each variant
for i in range(len(scale)):
    for j in range(len(ncells)):
        # Create  a folder for simulation and copy in the empirical potential file
        path = './Simulations/' + 'potential_' + str(potential_index+1) +  '/' + 'Scale_'+ str(scale[i]) + '/' + 'Cells_' + str(ncells[j]) + '/'
        if not os.path.exists(path):
            os.makedirs(path, mode=0o777)
        shutil.copy2('../Potentials/' + potential_file, path) 
        
        # Define values to replace in base input file
        replacements = {
            'ALATT':str(scale[i]*a_eq),
      		'NCELLS':str(ncells[j]),
            'POTENTIAL':potential_file,
            'PAIRSTYLE':pair_style
        }
        
        # Create a lammps file and use base file with string replacements to create unique variant
        inputLammpsFile = 'in_base.lmps'
        outputLammpsFile = path + 'in.lmps'        
        finLammps = open(inputLammpsFile, 'r').read()
        foutLammps = open(outputLammpsFile, 'w')
        out = replace_all(finLammps, replacements)
        foutLammps.write(out)
        foutLammps.close()

        lammps_executable = 'lmp'
        lammps_command = '-in in.lmps'

        with chdir(path):
            os.system(lammps_executable + ' ' + lammps_command)

### Gather results
Loop over all the simulations and gather the values of quantities of interest into a single file.

In [ ]:
output_path = './Simulations/' + 'potential_' + str(potential_index+1) + '/'

with chdir(output_path):

    output_file = 'results.txt'
    colwidth = 20
    fo = open(output_file,'w')
    fo.write('#Strain'.ljust(colwidth) + 'Lattice Param (A)'.ljust(colwidth) + 'Volume (A^3)'.ljust(colwidth) + 'Energy (eV)'.ljust(colwidth) + 'Pressure (GPa)'.ljust(colwidth) + '\n')

    for i in range(len(scale)):
        for j in range(len(ncells)):
            # Create  a folder for simulation and copy in the empirical potential file
            path = 'Scale_'+ str(scale[i]) + '/' + 'Cells_' + str(ncells[j]) + '/'
            
            log = lammps_logfile.File(path + 'log.lammps')

            step = log.get("Step")
            pe = log.get("TotEng")
            length = log.get("Lx")
            pressure = log.get("Pxx")   

            fo.write(
                str(scale[i]).ljust(colwidth) + 
                str(length[-1]/float(ncells[j])).ljust(colwidth) + 
                str(length[-1]**3).ljust(colwidth) + 
                str(pe[-1]).ljust(colwidth) + 
                str(pressure[-1]/10000.0).ljust(colwidth) + '\n'
                )
    fo.close()

### Load in the simulation results
Read the results of our simulations into a numpy array. Numpy makes this incredibly easy: one line is enough.

In [ ]:
data = np.loadtxt(output_path + 'results.txt', skiprows=1)

### Plot the data
The strain is in the first column of the array and the enrgy in the fourth column. We can easily plot a graph to examine the form of the data.

In [ ]:
fig,axes = plt.subplots(1,1, figsize=(4,4))
axes.plot(data[:,0],data[:,3], 'o-')
axes.set_xlabel('Scaling')
axes.set_ylabel('Energy (eV)')

### An equation of state
A common way of extracting the bulk modulus from data of this sort is to use the Birch-Murnaghan equation of state:
$$
    E(V) =  E_0 + \frac{9V_0B_0}{16}\left\lbrace \left[   \left( \frac{V_0}{V}  \right)^{\frac{2}{3}}  -1  \right]^3 B_0^\prime + \left[   \left( \frac{V_0}{V}  \right)^{\frac{2}{3}}  -1  \right]^2 \left[  6 -4 \left( \frac{V_0}{V}  \right)^{\frac{2}{3}}  \right]  \right\rbrace  \; .
$$
This relates the energy of our crystal to the volume. $E_0$ and $V_0$ are respectively the energy and volume of the crystal at the equilibrium volume (scaling = 1.0). $B_0$ is then the bulk modulus and $B_0'$ is its first derivative with respect to pressure.

We can encode this equation of state in a python function. Here I am setting the values of $V_0$ and $E_0$ with the values from the results file. Since $V_0$ and $V$ mostly appear in the quotient $V_0/V$, I am defining this in the third line of the function. The rest of the lines just encode the equation of state. It could all be done on a single line, but I've built up the result in stages to reduce the chances of errors and to make any debugging easier (maths isn't easy to read in code form). 

In [ ]:
def BM(V,B0,B0p):
    V0 = data[int((np.shape(data)[0]-1)/2),2]
    E0 = data[int((np.shape(data)[0]-1)/2),3]
    v = V0/V
    E = np.power(np.power(v,2/3)-1,3)*B0p
    E = E + np.power(np.power(v,2/3)-1,2) * (6 - 4*np.power(v,2/3))
    E = E * (9*V0*B0/16)
    E = E + E0
    return E

### Finding the best fit
Python has lots of useful tools in the `scipy` library. Here we are importing the `optimise` module and using it to find the values of $B_0$ and $B_0'$ that give the best fit to the data for the equation of state. These values will be stored in the variable `params`. Again, all it takes is a single line of code:

In [ ]:
params, params_covariance = optimize.curve_fit(BM, data[:,2], data[:,3])#,p0=[8200, 2])

We can check the result by plotting it against the data:

In [ ]:
fig,axes = plt.subplots(1,1, figsize=(4,4))
V_vals = np.linspace(7750,8850,100)
axes.plot(V_vals,BM(V_vals,params[0],params[1]), '-', label='Best fit')
axes.plot(data[:,2],data[:,3], 'o', label='Simulation')
axes.set_xlabel(r'Volume ($\AA^3$)')
axes.set_ylabel('Energy (eV)')
axes.legend()

The fit looks good.

### Assessing the result 
We now have our estimate of the bulk modulus $B_0$ from the simulation. To compare it to the experimental value we need to get it into the right units. Currently it is in electron volts per cubic angstrom. We'd like it in gigapascal. That's an easy conversion to make:

In [ ]:
eV = 1.602e-19
A = 1e-10
GPa = 1e9
print(f'Our estimate of the bulk modulus is {params[0]*eV/A**3/GPa:0.1f} GPa')


A quick estimate for Al from Google is $B_0=$ 75 GPa. This is a pretty reasonable match.


## Exercise 3: Finding the vacancy formation energy of model aluminium

### Change to the correct directory

In [ ]:
%cd /content/drive/MyDrive/cdt_training/MaterialsModellingPrimer/03-VacancyFormation

### Build and run simulations
Creates and runs a series of simulations in increasingly large simulation cells.

In [ ]:
ncells = [3,4,5,6,8]                            # Number of unit cells in each direction

if True:
    potential_file = potential_files[potential_index]
    pair_style = pair_styles[potential_index] 

    # Loop over number of unit cells in simulation
    # Create a folder and lammps input files for each variant
    for j in range(len(ncells)):
        # Create  a folder for simulation and copy in the empirical potential file
        path = './Simulations/' + 'potential_' + str(potential_index+1) +  '/' + 'Scale_'+ str(1.000) + '/' + 'Cells_' + str(ncells[j]) + '/'
        if not os.path.exists(path):
            os.makedirs(path, mode=0o777)
            shutil.copy2('../Potentials/' + potential_file, path)
        
        reference_atom = int((4*ncells[j]**3)/2)

        # Define values to replace in base input file
        replacements = {
            'ALATT':str(a_eq),
            'NCELLS':str(ncells[j]),
            'DISP':str((ncells[j] + 0.5)*a_eq/2.0),
            'POTENTIAL':potential_file,
            'PAIRSTYLE':pair_style,
            'REFATOM':str(reference_atom)
        }
        
        # Create a lammps file and use base file with string replacements to create unique variant
        inputLammpsFile = 'in_base.lmps'
        outputLammpsFile = path+'in.lmps'        
        finLammps = open(inputLammpsFile, 'r').read()
        foutLammps = open(outputLammpsFile, 'w')
        out = replace_all(finLammps, replacements)
        foutLammps.write(out)
        foutLammps.close()

        lammps_executable = 'lmp'
        lammps_command = '-in in.lmps'

        with chdir(path):
            os.system(lammps_executable + ' ' + lammps_command)

### Gather results
Gather the simulation results and write them to a file

In [ ]:
if True:
    potential_file = potential_files[potential_index]

    output_path = './Simulations/' + 'potential_' + str(potential_index+1) + '/'

    with chdir(output_path):

        output_file = 'results.txt'
        colwidth = 24
        fo = open(output_file,'w')
        fo.write('#NCells'.ljust(colwidth) + 'Number of atoms'.ljust(colwidth) + 'Volume (A^3)'.ljust(colwidth) + 'Energy (eV)'.ljust(colwidth) + 'Pressure (GPa)'.ljust(colwidth) + '\n')

        for j in range(len(ncells)):
            # Create  a folder for simulation and copy in the empirical potential file
            path = 'Scale_'+ str(1.000) + '/' + 'Cells_' + str(ncells[j]) + '/'
            log = lammps_logfile.File(path + 'log.lammps')

            step = log.get("Step")
            pe = log.get("TotEng")
            length = log.get("Lx")
            pressure = log.get("Pxx")
            atoms = log.get("Atoms")


            fo.write(
                str(ncells[j]).ljust(colwidth) + 
                str(atoms[-1]).ljust(colwidth) + 
                str(length[-1]**3).ljust(colwidth) + 
                str(pe[-1]).ljust(colwidth) + 
                str(pressure[-1]/10000.0).ljust(colwidth) + 
                '\n'
                )
        fo.close()

### Load in the simulation results
Read the results of our simulations into a numpy array. Numpy makes this incredibly easy: one line is enough.

We then plot the formation energy as a function of the simulation cell size.

In [ ]:
fo = open('Simulations/results.txt', 'w')
fo.write('Potential'.ljust(colwidth) + 'Number of atoms'.ljust(colwidth) + 'Vacancy Energy (eV)'.ljust(colwidth) + 'Pressure (GPa)'.ljust(colwidth) + '\n')

numrows = 1
numcols = 1
fig,axes = plt.subplots(numrows, numcols, figsize=(4,3))

if True:
    potential_file = potential_files[potential_index]
    output_path = './Simulations/' + 'potential_' + str(potential_index+1) + '/'
    data = np.loadtxt(output_path + 'results.txt', skiprows=1)

    Ev = data[:,3] - data[:,1]*E_eq
    
    fo.write(str(potential_index).ljust(colwidth) + str(data[-1,1]).ljust(colwidth) + str(Ev[-1]).ljust(colwidth) + str(data[-1,4]).ljust(colwidth) + '\n')

    axes.plot(data[:,0],Ev[:], 'bo-')
    axes.set_xlabel('Number of cells')  
    axes.set_ylabel('Energy (eV)')
    axes.set_title(potential_file)
    fig.tight_layout()
fo.close()

print(f'Our estimate of the Vacancy formation energy is is {Ev[-1]:0.6f} eV')


## Exercise 4: Set up a supercell of a grain boundary and determine the energy

### Change to the correct directory

In [ ]:
%cd /content/drive/MyDrive/cdt_training/MaterialsModellingPrimer/04-GrainBoundary

### Run the simulations

In [ ]:
if True:
    potential_file = potential_files[potential_index]
    pair_style = pair_styles[potential_index]

    # ------------- Set up a folder for the simulation
    path = './Simulations/' + 'potential_' + str(potential_index+1) +  '/'
    if not os.path.exists(path):
        print(path)
        os.makedirs(path, mode=0o777)

    # ------------- Copy the grain boundary structure
    shutil.copy2('lammps.txt', path)

    # Create  a folder for simulation and copy in the empirical potential file
    shutil.copy2('../Potentials/' + potential_file, path)

    # Define values to replace in base input file
    replacements = {
        'POTENTIAL':potential_file,
        'PAIRSTYLE':pair_style
        }
        
    # Create a lammps file and use base file with string replacements to create unique variant
    inputLammpsFile = 'in_base.lmps'
    outputLammpsFile = path+'in.lmps'        
    finLammps = open(inputLammpsFile, 'r').read()
    foutLammps = open(outputLammpsFile, 'w')
    out = replace_all(finLammps, replacements)
    foutLammps.write(out)
    foutLammps.close()

    lammps_executable = 'lmp'
    lammps_command = '-in in.lmps'

    with chdir(path):
        os.system(lammps_executable + ' ' + lammps_command)

### Check the results from the simulations

These cells are for use after the simulation has run, to examine the output.

Read in data from the logfile and calculate the grain boundary energy.

In [ ]:
ev = 1.602E-19 #(J)
A = 1E-10 #(m)
natoms = 19120

if True:
    potential_file = potential_files[potential_index]
    path = './Simulations/' + 'potential_' + str(potential_index+1) +  '/'
    log = lammps_logfile.File(path + 'log.lammps')

    step = log.get("Step")
    pe = log.get("PotEng")
    ly = log.get("Ly")
    lz = log.get("Lz")
    pressure = log.get("Pxx")

    Egb = (pe[-1] - natoms*E_eq)/ly[-1]/lz[-1]

    print(f'Our estimate of the grain boundary energy is {Egb:0.4f} eV/A^2')  

## Exercise 5: Find the energy of an unrelaxed interstitial

### Change to the correct directory

In [ ]:
%cd /content/drive/MyDrive/cdt_training/MaterialsModellingPrimer/05-UnrelaxedInterstitial

In [ ]:
ncells = [6,8,10,12]                            # Number of unit cells in each direction

if(True):
    potential_file = potential_files[potential_index]
    pair_style = pair_styles[potential_index] 

    # Loop over number of unit cells in simulation
    # Create a folder and lammps input files for each variant
    for j in range(len(ncells)):
        # Create  a folder for simulation and copy in the empirical potential file
        path = './Simulations/' + 'potential_' + str(potential_index+1) +  '/' + 'Scale_'+ str(1.000) + '/' + 'Cells_' + str(ncells[j]) + '/'
        if not os.path.exists(path):
            os.makedirs(path, mode=0o777)
            shutil.copy2('../Potentials/' + potential_file, path)
        
        reference_atom = int((4*ncells[j]**3)/2)

        # Define values to replace in base input file
        replacements = {
            'ALATT':str(a_eq),
            'NCELLS':str(ncells[j]),
            'DISP':str((ncells[j] - 0.5)*a_eq/2.0),
            'POTENTIAL':potential_file,
            'PAIRSTYLE':pair_style,
            'REFATOM':str(reference_atom)
        }
        
        # Create a lammps file and use base file with string replacements to create unique variant
        inputLammpsFile = 'in_base.lmps'
        outputLammpsFile = path+'in.lmps'        
        finLammps = open(inputLammpsFile, 'r').read()
        foutLammps = open(outputLammpsFile, 'w')
        out = replace_all(finLammps, replacements)
        foutLammps.write(out)
        foutLammps.close()

        lammps_executable = 'lmp'
        lammps_command = '-in in.lmps'

        with chdir(path):
            os.system(lammps_executable + ' ' + lammps_command)

### Read in the simulation results

In [ ]:
if True:
    potential_file = potential_files[potential_index]

    output_path = './Simulations/' + 'potential_' + str(potential_index+1) + '/'

    with chdir(output_path):

        output_file = 'results.txt'
        colwidth = 24
        fo = open(output_file,'w')
        fo.write('#NCells'.ljust(colwidth) + 'Number of atoms'.ljust(colwidth) + 'Volume (A^3)'.ljust(colwidth) + 'Energy (eV)'.ljust(colwidth) + 'Pressure (GPa)'.ljust(colwidth) + '\n')

        for j in range(len(ncells)):
            # Create  a folder for simulation and copy in the empirical potential file
            path = 'Scale_'+ str(1.000) + '/' + 'Cells_' + str(ncells[j]) + '/'
            log = lammps_logfile.File(path + 'log.lammps')

            step = log.get("Step")
            pe = log.get("TotEng")
            length = log.get("Lx")
            pressure = log.get("Pxx")
            atoms = log.get("Atoms")


            fo.write(
                str(ncells[j]).ljust(colwidth) + 
                str(atoms[-1]).ljust(colwidth) + 
                str(length[-1]**3).ljust(colwidth) + 
                str(pe[-1]).ljust(colwidth) + 
                str(pressure[-1]/10000.0).ljust(colwidth) + 
                '\n'
                )
        fo.close()

### Calculate the defect energy


In [ ]:
fo = open('Simulations/results.txt', 'w')
fo.write('Potential'.ljust(colwidth) + 'Number of atoms'.ljust(colwidth) + 'Vacancy Energy (eV)'.ljust(colwidth) + 'Pressure (GPa)'.ljust(colwidth) + '\n')

numrows = 1
numcols = 1
fig,axes = plt.subplots(numrows, numcols, figsize=(4,3))

if True:
    potential_file = potential_files[potential_index]
    output_path = './Simulations/' + 'potential_' + str(potential_index+1) + '/'
    data = np.loadtxt(output_path + 'results.txt', skiprows=1)

    Ev = data[:,3] - data[:,1]*E_eq
    
    fo.write(str(potential_index).ljust(colwidth) + str(data[-1,1]).ljust(colwidth) + str(Ev[-1]).ljust(colwidth) + str(data[-1,4]).ljust(colwidth) + '\n')

    axes.plot(data[:,0],Ev[:], 'bo-')
    axes.set_xlabel('Number of cells')  
    axes.set_ylabel('Energy (eV)')
    axes.set_title(potential_file)
    fig.tight_layout()
fo.close()

print(f'Our estimate of the defect energy is {Ev[-1]:0.3f} eV')